<a href="https://colab.research.google.com/github/JoaoVLima/ActualSimplePDFValidator/blob/main/TDE01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PUCPR
## **João Victor de Lima**
### TDE01 - Construção de Interpretadores
**Desenvolver um programa que analise documentos no
formato simplepdf**, uma versão textual simplificada inspirada na estrutura interna de arquivos pdf.
O programa deverá **processar documentos simplepdf**, **validar sua estrutura segundo as regras formais da linguagem que aqui será definida**, **gerar estatísticas** e **transformações do conteúdo**.

# Explicação

## Formato SimplePDF

O SimplePDF é combosto por:

1. **Cabeçalho**: Todo documento começa com uma linha de cabeçalho
```
%SPDF-1.0
```

2. **Objetos**: O documento consiste em objetos numerados no formato
```
obj_id obj_gen obj
<< conteudo >>
endobj
```
No qual:
  - **obj_id** é um número inteiro único que identifica o objeto
  - **obj_gen** é um número de geração
(normalmente 0)

3. **Tipos de dados básicos**:
  - **Números**: inteiros (123) ou decimais (123.45)
  - **Strings**: delimitadas por parênteses: (Este é um texto)
  - **Nomes**: começam com /: /Nome
  - **Booleanos**: true ou false
  - **Null**: null
  - **Referências**: indicam referência a outro objeto: obj_id obj_gen R

4. **Estruturas de dados**:
  - **Dicionários**: pares chave/valor no formato: /Chave valor
  - **Arrays**: sequências de valores delimitadas por [ e ]
  - **Streams**: Blocos de dados entre stream e endstream

5. **Estrutura hierárquica**:
  - **Objeto catálogo (root)**: Define a estrutura do documento
  - **Objeto de páginas**: Mantém lista de páginas
  - **Objetos de página**: Contêm conteúdo e propriedades de cada página
  - **Objetos de conteúdo**: Contêm texto e elementos gráficos


6. **Comentários**:
  Linhas que começam com % (exceto o cabeçalho) são comentários

7. **Cross-reference table (xref)**: Tabela que lista a posição de cada objeto no arquivo:
```
xref
0 3
0000000000 65535 f
0000000010 00000 n
0000000079 00000 n
```

8. **Trailer**: Informações finais do documento:
```
trailer
<< /Size 3 /Root 1 0 R >>
startxref
183
%%EOF
```

## Funcionalidades Requeridas do Programa

O programa deve implementar as seguintes funcionalidades:

1. **Validação Estrutural**:
  - Verificar se o cabeçalho SimplePDF está correto
  - Validar a sintaxe de cada objeto e suas propriedades
  - Verificar se todas as referências apontam para objetos existentes
  - Validar a estrutura hierárquica (catálogo → páginas → página)
  - Verificar consistência da tabela xref
  - Validar o trailer

2. **Extração de Informações**:  
  - Conteúdo textual de cada página
  - Metadados do documento (título, autor, data)
  - Estrutura de páginas (quantidade, tamanho)
  - Fontes utilizadas
  - Estatísticas de objetos por tipo

3. **Transformações**:
  - Extração de todo texto para formato TXT;
  - Geração de sumário baseado na estrutura
  - Listagem hierárquica de objetos e suas dependências
  - Conversão de coordenadas de texto para formato mais legível

4. **Análise Avançada**:
  - Detecção de objetos não referenciados
  - Identificação de ciclos de referência
  - Análise de eficiência de armazenamento
  - Sugestões de otimização da estrutura

5. **O programa deve ler dois arquivos**:
  - O documento SimplePDF a ser analisado
  - Um arquivo de configuração que define parâmetros de análise e transformação

## Arquivo de Configuração

O arquivo sera disponibilizado com o mesmo nome do arquivo pdf, porem com o formato .py

Teste01.pdf
```
%SPDF-1.0
...
```

Teste01.py
```
extrair_texto=True
gerar_sumario=True
detectar_ciclos=True
nivel_detalhe='Completo'
validar_xref=True
```

Opções:
- **extrair_texto** (bool): Habilita a extração de texto do documento.
- **gerar_sumario** (bool): Gera um sumário do conteúdo do documento.
- **detectar_ciclos** (bool): Detecta possíveis referências cíclicas no documento.
- **nivel_detalhe** (str): Define o nível de detalhe da análise ('Completo', 'Basico', 'Nulo').
- **validar_xref** (bool): Valida as referências cruzadas dentro do documento.

## Saída do Programa

O programa deve gerar um relatório com as seguintes seções:
```
VALIDAÇÃO:
[OK ou ERRO] Estrutura geral
[OK ou ERRO] Sintaxe de objetos
[OK ou ERRO] Referências
[OK ou ERRO] Tabela xref
[Detalhes de erros, se houver]

ESTATÍSTICAS:
Total de objetos: X
Objetos por tipo: Catalog=1, Pages=1, Page=Y, Font=Z, ...
Total de páginas: Y
Tamanho do documento: W bytes
Overhead estrutural: V bytes (P%)

CONTEÚDO:
Título: [título do documento]
Autor: [autor do documento]
Data de criação: [data]
Texto extraído: [primeiros 200 caracteres...]

ÁRVORE DE OBJETOS:
1: Catalog
  +- 2: Pages
    +- 3: Page
    +- 4: Font
    +- 5: Contents (stream)
6: Metadata

ANÁLISE AVANÇADA:
[Resultados de análises específicas definidas no arquivo de configuração]
```

## AS TRANSFORMAÇÕES ESPECIFICADAS

## Exemplos

Exemplo 1
```
%SPDF-1.0

1 0 obj
<< /Type /Catalog
/Pages 2 0 R
/Metadata 7 0 R
/Outlines 8 0 R
>>
endobj

2 0 obj
<< /Type /Pages
/Kids [3 0 R 4 0 R]
/Count 2
>>
endobj

3 0 obj
<< /Type /Page
/Parent 2 0 R
/Resources << /Font << /F1 5 0 R >> >>
/MediaBox [0 0 612.0 792.0]
/CropBox [10.5 10.5 601.5 781.5]
/Rotate 0
/Contents 6 0 R
>>
endobj

4 0 obj
<< /Type /Page
/Parent 2 0 R
/Resources << /Font << /F1 5 0 R >> >>
/MediaBox [0 0 612.0 792.0]
/CropBox [10.5 10.5 601.5 781.5]
/Rotate 0
/Contents 9 0 R
>>
endobj

5 0 obj
<< /Type /Font
/Subtype /Type1
/BaseFont /Helvetica
/FontDescriptor 10 0 R
>>
endobj

6 0 obj
<< /Length 175 >>
stream
BT
/F1 24 Tf
100 700 Td
(Relatório Financeiro: Primeiro Trimestre) Tj
/F1 12 Tf
0 -50 Td
(Total de vendas: 1423 unidades) Tj
0 -20 Td
(Receita total: 158432.75 reais) Tj
ET
endstream
endobj

7 0 obj
<< /Title (Relatório Financeiro Trimestral)
/Author (Departamento Financeiro)
/CreationDate (D:20230418090000)
/Keywords (finanças, relatório, trimestral)
/Version 1.2
/PageCount 2
>>
endobj

8 0 obj
<< /Type /Outlines
/Count 3
/First 11 0 R
/Last 13 0 R
>>
endobj

9 0 obj
<< /Length 322 >>
stream
BT
/F1 18 Tf
100 700 Td
(Detalhamento por Categoria) Tj
/F1 12 Tf
0 -40 Td
(Produto A: 523 unidades, valor unitário 89.99, total 47054.77) Tj
0 -20 Td
(Produto B: 347 unidades, valor unitário 125.50, total 43548.50) Tj
0 -20 Td
(Produto C: 553 unidades, valor unitário 122.65, total 67825.45) Tj
0 -40 Td
(Média de vendas por produto: 474.33 unidades) Tj
ET
endstream
endobj

10 0 obj
<< /Type /FontDescriptor
/FontName /Helvetica
/Flags 32
/FontBBox [-166 -225 1000 931]
/ItalicAngle 0
/Ascent 718
/Descent -207
/CapHeight 718
/StemV 88
/MissingWidth 278
>>
endobj

11 0 obj
<< /Title (Resumo Executivo)
/Parent 8 0 R
/Next 12 0 R
/Dest [3 0 R /FitH 792.0]
>>
endobj

12 0 obj
<< /Title (Detalhamento de Vendas)
/Parent 8 0 R
/Prev 11 0 R
/Next 13 0 R
/Dest [4 0 R /FitH 792.0]
>>
endobj

13 0 obj
<< /Title (Projeções Futuras)
/Parent 8 0 R
/Prev 12 0 R
/Dest [4 0 R /XYZ 100 450 1.25]
>>
endobj

14 0 obj
<< /Type /Metadata
/Subtype /XML
/Length 340
>>
stream
<?xpacket begin='' id='W5M0MpCehiHzreSzNTczkc9d'?>
<x:xmpmeta xmlns:x='adobe:ns:meta/'>
<rdf:RDF xmlns:rdf='http://www.w3.org/1999/02/22-rdf-syntax-ns#'>
<rdf:Description rdf:about=''
xmlns:pdf='http://ns.adobe.com/pdf/1.3/'>
<pdf:Producer>SimplePDF Generator 1.0</pdf:Producer>
</rdf:Description>
</rdf:RDF>
</x:xmpmeta>
<?xpacket end='w'?>
endstream
endobj
xref
0 15
0000000000 65535 f
0000000010 00000 n
0000000099 00000 n
0000000163 00000 n
0000000344 00000 n
0000000525 00000 n
0000000624 00000 n
0000000852 00000 n
0000001048 00000 n
0000001126 00000 n
0000001500 00000 n
0000001712 00000 n
0000001818 00000 n
0000001940 00000 n
0000002052 00000 n
trailer
<< /Size 15
/Root 1 0 R
/Info 7 0 R
>>
startxref
2500
%%EOF
```

Exemplo 2
```

```

# Código